# Topological Data Analysis with Python and the Gudhi Library 

# Introduction to simplex trees ([download](https://raw.githubusercontent.com/GUDHI/TDA-tutorial/master/Tuto-GUDHI-simplex-Trees.ipynb))

**Authors** : F. Chazal and B. Michel

TDA typically aims at extracting topological signatures from a point cloud in $\mathbb R^d$ or in a general metric space. By studying the topological of the point clouds, we actually mean studying the topology of unions of balls centered at the point cloud (offsets). However, non-discrete sets such as offsets, and also continuous mathematical shapes like curves, surfaces and more generally manifolds, cannot easily be encoded as finite discrete structures. [Simplicial complexes](https://en.wikipedia.org/wiki/Simplicial_complex) are therefore used in computational geometry to approximate such shapes.

A simplicial complex is a set of [simplices](https://en.wikipedia.org/wiki/Simplex), they can be seen as higher dimensional generalization of graphs. They are mathematical objects that are both topological and combinatorial, a property making them particularly useful for TDA. Here is an exemple of simplicial complex :

![title](Images/Pers14.PNG)
 
A Filtration is a increasing sequence of sub-complexes of a simplicial complex $K$, it can be seen as ordering the simplices included in the complex. Indeed, simpicial complexes often come with a specific order, as for [Vietoris-Rips complexes](https://en.wikipedia.org/wiki/Vietoris%E2%80%93Rips_complex), [Cech complexes](https://en.wikipedia.org/wiki/%C4%8Cech_complex) and [alpha complexes](https://en.wikipedia.org/wiki/Alpha_shape#Alpha_complex). 

In [1]:
from IPython.display import Image
from os import chdir
import numpy as np
import gudhi as gd
import matplotlib.pyplot as plt

In Gudhi, filtered simplicial complexes are encoded through a data structure called simplex tree. 
![CSexemple](http://gudhi.gforge.inria.fr/python/latest/_images/Simplex_tree_representation.png)

This notebook illustrates the use of simplex tree to represent simplicial complexes from data points.

See the [Python Gudhi documentation](http://gudhi.gforge.inria.fr/python/latest/simplex_tree_ref.html#) for more details on simplex trees.

### My first simplex tree

Let's create our first simplicial complex, represented by a simplex tree :

In [2]:
st = gd.SimplexTree()

The `st` object has class `SimplexTree`. For now `st` is an empty simplex tree.

The `SimplexTree` class has several usufull methods for the practice of TDA, for instance for defining new types of simplicial complexes from existing ones. 

The `insert()` method can be used to insert simplices in the simplex tree. In the simplex tree, vertices are represented by integers, an edge is represented by the list of its two integers corresponding to the vertices, a triangle by three integers etc. 

Now we insert three edges:

In [3]:
st.insert([0,1])
st.insert([1,2])
st.insert([3,1])

True

If the simplex is already in the filtration then the `insert()` method outputs the boolean `False`.

In [4]:
st.insert([3,1])

False

We obtain the list of all the simplices in the simplex tree with the `get_filtration()` method : 

In [5]:
st_list = st.get_filtration() 

The output `st_list` is a list and we thus we can iterate on its elements. Each element in the list is a tuple that contains a simplex and its filtration value.

In [6]:
for splx in st_list :
    print(splx)

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([3], 0.0)
([1, 3], 0.0)


Intuitively, the filtration value of a simplex in a filtered complex corresponds to "when" the simplex appears in the filtration. By default, with the `insert()` method the simplex has a filtration value equal to 0.

Notice that inserting an edge automatically insert its vertices (if they were not already in the complex) in order to satisfy the inclusion property of a filtered complex: any simplex with filtration value $t$ has all its faces in the filtered complex, with filtration values smaller than $t$.

### Simplex tree description

The dimension of the simplex tree is given by the `dimension()` method, it corresponds to the largest dimension of the simplices in the simplex tree:

In [7]:
st.dimension()

1

It is possible to compute  the number of vertices in the simplex tree:

In [8]:
st.num_vertices()

4

The number of simplices in the simplex tree is given by

In [9]:
st.num_simplices()

7

The [d-skeleton](https://en.wikipedia.org/wiki/N-skeleton) for every dimension $d$ can be also computed with the `get_skeleton_tree()` method:

In [10]:
print(st.get_skeleton(1))

[([0, 1], 0.0), ([0], 0.0), ([1, 2], 0.0), ([1, 3], 0.0), ([1], 0.0), ([2], 0.0), ([3], 0.0)]


One can also test if a simplex is already in the filtration with the `find()` method:

In [11]:
st.find([2, 4])

False

### Filtration values

We can insert simplices for a given filtration value. Below we insert in the simplex tree three triangles at three different filtrations values:

In [12]:
st.insert([0,1,2],filtration=0.1)
st.insert([1,2,3],filtration=0.2)
st.insert([0,1,3],filtration=0.4)
st_list  = st.get_filtration() 

for splx in st_list :
    print(splx)

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([3], 0.0)
([1, 3], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([2, 3], 0.2)
([1, 2, 3], 0.2)
([0, 3], 0.4)
([0, 1, 3], 0.4)


If we add a new simplex with a given filtration values, all its faces that were not in the complex before are added with the same filtration value : here the edge [0, 3] has been also inserted and it takes the filtration value 0.4.

The `assign_filtration()` method can be used to assign a filtration value to a simplex that is 
already in the filtration.

In [13]:
st.assign_filtration([3],filtration=0.8)
st_list = st.get_filtration()
for splx in st_list:
    print(splx)   

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([1, 3], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([2, 3], 0.2)
([1, 2, 3], 0.2)
([0, 3], 0.4)
([0, 1, 3], 0.4)
([3], 0.8)


However this simplex tree is not a filtered simplicial complex anymore because the filtration value of the vertex [3] is higher then the filtration value of the edge [2 3]. We can use the `make_filtration_non_decreasing()` method to solve the problem:

In [14]:
st.make_filtration_non_decreasing()
st_list = st.get_filtration()
for splx in st_list:
    print(splx)  

([0], 0.0)
([1], 0.0)
([0, 1], 0.0)
([2], 0.0)
([1, 2], 0.0)
([0, 2], 0.1)
([0, 1, 2], 0.1)
([3], 0.8)
([0, 3], 0.8)
([1, 3], 0.8)
([0, 1, 3], 0.8)
([2, 3], 0.8)
([1, 2, 3], 0.8)


The `filtration()` function  returns the filtration level of a given simplex in the filtration :

In [15]:
st.filtration([2,3])

0.8